In [ ]:
import numpy as np
import pandas as pd
import pickle

# 1. Data
- Load data from pickle file
- split the data into independent and dependent
- split to train and test set

In [ ]:
data = pickle.load(open('data_face_features.pickle',mode='rb'))

In [ ]:
X = np.array(data['data']) 
y = np.array(data['label']) 

In [ ]:
X.shape , y.shape

((1654, 1, 128), (1654,))

In [ ]:
X = X.reshape(-1,128)
X.shape

(1654, 128)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1323, 128), (331, 128), (1323,), (331,))

# 2. Train Machine Learning

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

### Logistic Regression

In [ ]:
model_logistic = LogisticRegression()
model_logistic.fit(x_train,y_train) 

LogisticRegression()

In [ ]:
def get_report(model, x_train,y_train,x_test,y_test):
    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)

    # accuracy score
    acc_train = accuracy_score(y_train,y_pred_train)
    acc_test = accuracy_score(y_test,y_pred_test)

    # f1 score
    f1_score_train = f1_score(y_train,y_pred_train,average='macro')
    f1_score_test = f1_score(y_test,y_pred_test,average='macro')


    print('Accuracy Train = %0.2f'%acc_train)
    print('Accuracy Test = %0.2f'%acc_test)
    print('F1 Score Train = %0.2f'%f1_score_train)
    print('F1 Score Test = %0.2f'%f1_score_test)

In [ ]:
get_report(model_logistic,x_train,y_train,x_test,y_test)

Accuracy Train = 0.73
Accuracy Test = 0.69
F1 Score Train = 0.72
F1 Score Test = 0.68


### Support Vector Machines

In [ ]:
model_svc = SVC(probability=True)
model_svc.fit(x_train,y_train)

SVC(probability=True)

In [ ]:
get_report(model_svc,x_train,y_train,x_test,y_test)

Accuracy Train = 0.83
Accuracy Test = 0.71
F1 Score Train = 0.83
F1 Score Test = 0.71


### Random Forest

In [ ]:
model_rf = RandomForestClassifier(n_estimators=10,)
model_rf.fit(x_train,y_train)

RandomForestClassifier(n_estimators=10)

In [ ]:
get_report(model_rf,x_train,y_train,x_test,y_test)

Accuracy Train = 1.00
Accuracy Test = 0.56
F1 Score Train = 1.00
F1 Score Test = 0.56


## Voting Classifier

In [ ]:
model_voting = VotingClassifier(estimators=[
    ('logistic',LogisticRegression()),
    ('svm',SVC(probability=True)),
    ('rf',RandomForestClassifier())
], voting='soft',weights=[2,3,1])

In [ ]:
model_voting.fit(x_train,y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier())],
                 voting='soft', weights=[2, 3, 1])

In [ ]:
get_report(model_voting,x_train,y_train,x_test,y_test)

Accuracy Train = 0.88
Accuracy Test = 0.70
F1 Score Train = 0.88
F1 Score Test = 0.70


# 3. Parameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
model_grid = GridSearchCV(model_voting,
                         param_grid={
                             'svm__C':[3,5,7,10],
                             'svm__gamma':[0.1,0.3,0.5],
                             'rf__n_estimators':[5,10,20],
                             'rf__max_depth':[3,5,7],
                             'voting':['soft']
                         },scoring='accuracy',cv=3,n_jobs=1,verbose=2)

In [ ]:
model_grid.fit(x_train,y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.6s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.6s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.6s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   0.6s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   0.6s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   0.6s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.5s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.5s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.5s
[

GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier())],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [ ]:
model_best_estimator = model_grid.best_estimator_

In [ ]:
model_grid.best_score_

0.7059712773998488

# 4. Save Model

In [ ]:
pickle.dump(model_best_estimator,open('./models/machinelearning_face_person_identity.pkl',mode='wb'))